<a href="https://colab.research.google.com/github/RogerHeederer/ML_practice_MyFirstML_B/blob/master/CNN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source reference : https://github.com/wikibook/machine-learnin

**위 레퍼런스를 참조하여 공부하였으며, 필요하다고 생각되는 부분에 추가적인 설명 ,코드 추가, 그림 삽입 등을 덧붙였습니다..**


In [ ]:
from IPython.display import Image

In [ ]:

Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/practice_cnn.png", width=800, height=200)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

tf.random.set_seed(1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
x_train[0][8], x_train[0][8][7]

(array([  0,   0,   0,   0,   0,   0,   0,  18, 219, 253, 253, 253, 253,
        253, 198, 182, 247, 241,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0], dtype=uint8), 18)

In [ ]:
y_train[0]

5

**훈련 데이터 6만개/ 테스트 데이터 1만개**

**데이터 형태는 28*28**

In [ ]:
print(x_train.shape[0], x_train.shape[1], x_train.shape[2])

60000 28 28


In [ ]:
import numpy as np
x_train = np.reshape(x_train, (60000, 28, 28, 1))
x_test = np.reshape(x_test, (10000, 28, 28, 1))

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


**데이터 정규화**

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

In [ ]:
print(x_train[0][8], x_train[0][8][7]) #위 32번째 라인과 같은 값

[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.07058824]
 [0.85882354]
 [0.99215686]
 [0.99215686]
 [0.99215686]
 [0.99215686]
 [0.99215686]
 [0.7764706 ]
 [0.7137255 ]
 [0.96862745]
 [0.94509804]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]] [0.07058824]


In [ ]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train[0] #레이블 값 5

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

CNN 텐서플로우 구현

In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5,5), activation='relu', input_shape=(28,28,1), padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2))) #28*28은 maxpooling 후 14*14로 변환됨.
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2))) # 14*14는 maxpooling 후 7*7로 변환됨

#마지막 단은 Multi Layer Perceptron으로 히든 레이어 쌓아서, classification 한다
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               200832    
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

** weight / bias파라미터 개수 계산은 다음과 같다**

 conv2d : 5*5(필터) X 16(필터갯수) + 16(Bias) = 416

 conv2d_1 : 5*5(필터) X 16(위의 필터로 부터 나온 feature map 갯수) X 32(새로운 필터) + 32(Bias) = 12832

  dense : 1568(Flatten input) * 128(히든레이어1 노드 수) + 128 (bias) = 200832

  dense_1 : 128(히든레이어 1 출력 노드) * 10(소프트맥스 분류 값) + 10(bias) = 1290

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only = True)]

In [ ]:
model.fit(x_train, y_train,
          batch_size=500,
          epochs=5,
          verbose=1,
          validation_split = 0.1,
          callbacks=callbacks)

Epoch 1/5
108/108 [==============================] - 50s 464ms/step - loss: 0.4869 - accuracy: 0.8704 - val_loss: 0.1215 - val_accuracy: 0.9658
Epoch 2/5
108/108 [==============================] - 50s 462ms/step - loss: 0.1075 - accuracy: 0.9681 - val_loss: 0.0771 - val_accuracy: 0.9787
Epoch 3/5
108/108 [==============================] - 50s 463ms/step - loss: 0.0713 - accuracy: 0.9785 - val_loss: 0.0571 - val_accuracy: 0.9838
Epoch 4/5
108/108 [==============================] - 50s 460ms/step - loss: 0.0543 - accuracy: 0.9836 - val_loss: 0.0478 - val_accuracy: 0.9873
Epoch 5/5
108/108 [==============================] - 50s 461ms/step - loss: 0.0448 - accuracy: 0.9862 - val_loss: 0.0414 - val_accuracy: 0.9885


In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 3s 11ms/step - loss: 0.0383 - accuracy: 0.9867
Test loss: 0.03829098120331764
Test accuracy: 0.9866999983787537
